In [1]:
#importing packages needed for analysis
import os
import numpy as np
import pandas as pd
import math
from pandas import DataFrame

path = os.getcwd()
#print(path)

load_dur = pd.read_csv('../outputs/load_long_format.csv')
solar_dur = pd.read_csv('../outputs/solar_long_format.csv')
wind_dur = pd.read_csv('../outputs/wind_long_format.csv')

## UNCOMMENT WHICH PROFILE TO BE USED
x = load_dur
x_name = 'load'
x_name2 = 'Load'
x_column = 'Load'

#x = solar_dur
#x_name = 'solar'
#x_name2 = 'Solar_Gen'
#x_column = 'TRG6'

#x = wind_dur
#x_name = 'wind'
#x_name2 = 'Wind_Gen'
#x_column = 'TRG4'

#this code creates an output directory in the parent director, if one does not exist yet
#Note: this is where all of the output files will be written, since outputs are large this saves space in git
path = os.getcwd()
parent = os.path.dirname(path)
outputs_dir = parent+'\outputs'
if not os.path.exists(outputs_dir):
    os.makedirs(outputs_dir)
print('output files are written out in parent directory: '+outputs_dir)

outputs_x = outputs_dir+'/'+x_name
if not os.path.exists(outputs_x):
    os.makedirs(outputs_x)
print('output files are written out in parent directory: '+outputs_x)

x = x[['Region','R_Group','R_Subgroup','Season','Month','Day','Hour',x_column]]
years = pd.read_csv('inputs/years.csv').dropna()
#print(x.head())

unique_r = pd.Series(load_dur['Region'].unique()).dropna()
#print(unique_r)
reg_count = unique_r.shape[0]

output files are written out in parent directory: C:\Users\tgoforth\Documents\IPM temporal resolution project\outputs
output files are written out in parent directory: C:\Users\tgoforth\Documents\IPM temporal resolution project\outputs/load


In [2]:
import datetime
daydata = pd.read_csv('inputs/days_365.csv')
#print(daydata.tail())
daydata = daydata.drop(columns='Month')
x2 = pd.merge(x,daydata,on=['Day'],how='left')
#print(x2.tail())

#sets the year for each region
x2['Year']=2011
x2.loc[x2['R_Group'] == 'ERC', 'Year'] = 2016
#print(x2.head())

#Creates a date column
x2 = x2.rename(columns={'Day':'DOY','DayofMo':'Day'})
x2['Date']=pd.to_datetime(x2[['Year', 'Month', 'Day']], errors='coerce')
#print(x2.tail())

#convert date to a datetime type 
#x2['Date'] = pd.to_datetime(x2['Date'])
x2['DOW'] = x2['Date'].dt.weekday

#check if it is a weekday or not 
weekday = pd.read_csv('inputs/weekday.csv')
x2 = pd.merge(x2,weekday,on='DOW',how='left')
print(x2.head())

     Region R_Group R_Subgroup  Season  Month  DOY  Hour   Load  Day  Year  \
0  ERC_REST     ERC       REST  winter      1    1     1  34807    1  2016   
1  ERC_REST     ERC       REST  winter      1    2     1  34716    2  2016   
2  ERC_REST     ERC       REST  winter      1    3     1  34736    3  2016   
3  ERC_REST     ERC       REST  winter      1    4     1  35914    4  2016   
4  ERC_REST     ERC       REST  winter      1    5     1  33845    5  2016   

        Date  DOW  Weekday  
0 2016-01-01    4     True  
1 2016-01-02    5    False  
2 2016-01-03    6    False  
3 2016-01-04    0     True  
4 2016-01-05    1     True  


## Case 1: Monthly, single day type, 24 hours (288 segments)
#### Methodology: Using groupby function to group first by month, then by 24 hours


In [4]:
case1_x = x2.copy()

aggregations = {x_column:['count',sum,'mean']}
case1 = case1_x.groupby(['Region','Month','Hour'],as_index=False).agg(aggregations)
case1.columns = case1.columns.droplevel(0)
case1.columns = ['Region','Month','Hour','Hour_Tot','Tot','Avg']
print(case1.head())
print('number of segments in dataset =',case1.shape[0]/reg_count)
case1.to_csv('../outputs/'+x_name+'/'+x_name+'_segments_1dt_mon_24hr.csv')
print()

case1_x2 = pd.merge(case1_x,case1,on=['Region','Month','Hour'],how='left')
case1_x2 = case1_x2.sort_values(['Region',x_column])
print(case1_x2.head(3))
print('number of rows in dataset =',case1_x2.shape[0])
case1_x2.to_csv('../outputs/'+x_name+'/'+x_name+'_8760_1dt_mon_24hr.csv')


     Region  Month  Hour  Hour_Tot      Tot           Avg
0  ERC_REST      1     1        31  1042446  33627.290323
1  ERC_REST      1     2        31  1051661  33924.548387
2  ERC_REST      1     3        31  1079739  34830.290323
3  ERC_REST      1     4        31  1144492  36919.096774
4  ERC_REST      1     5        31  1239385  39980.161290
number of segments in dataset = 288.0

         Region R_Group R_Subgroup    Season  Month  DOY  Hour   Load  Day  \
46075  ERC_REST     ERC       REST  shoulder      3   86     3  26989   27   
23088  ERC_REST     ERC       REST  shoulder      4   94     2  27006    4   
46061  ERC_REST     ERC       REST  shoulder      3   72     3  27025   13   

       Year       Date  DOW  Weekday  Hour_Tot     Tot           Avg  
46075  2016 2016-03-27    6    False        31  886708  28603.483871  
23088  2016 2016-04-04    0     True        30  870591  29019.700000  
46061  2016 2016-03-13    6    False        31  886708  28603.483871  
number of rows i

## Case 2: Season, single day-type, 24 hours (72 segments)
#### Methodology: Use groupby function to group by season and hour

In [5]:
case2_x = x2.copy()
#case2_seasons = pd.read_csv('inputs/season_bimonthly.csv')
#case2_seasons = case2_seasons.drop(['bimonthly'], axis=1)
#case2_seasons = case2_seasons.rename(columns={'seasonal':'Season_Group','month':'Month'})
#print(case2_x.head())
#case2_x = pd.merge(case2_x, case2_seasons, on='Month', how='left')

aggregations = {x_column:['count',sum,'mean']}
case2 = case2_x.groupby(['Region','Season','Hour'],as_index=False).agg(aggregations)
case2.columns = case2.columns.droplevel(0)
case2.columns = ['Region','Season','Hour','Hour_Tot','Tot','Avg']
print(case2.head())
print('number of segments in dataset =',case2.shape[0]/reg_count)
case2.to_csv('../outputs/'+x_name+'/'+x_name+'_segments_1dt_sea_24hr.csv')
print()

case2_x2 = pd.merge(case2_x,case2,on=['Region','Season','Hour'],how='left')
case2_x2 = case2_x2.sort_values(['Region',x_column])
print(case2_x2.head(3))
print('number of rows in dataset =',case2_x2.shape[0])
case2_x2.to_csv('../outputs/'+x_name+'/'+x_name+'_8760_1dt_sea_24hr.csv')


     Region    Season  Hour  Hour_Tot      Tot           Avg
0  ERC_REST  shoulder     1       122  3635975  29803.073770
1  ERC_REST  shoulder     2       122  3587428  29405.147541
2  ERC_REST  shoulder     3       122  3603145  29533.975410
3  ERC_REST  shoulder     4       122  3731428  30585.475410
4  ERC_REST  shoulder     5       122  3992001  32721.319672
number of segments in dataset = 72.0

         Region R_Group R_Subgroup    Season  Month  DOY  Hour   Load  Day  \
46075  ERC_REST     ERC       REST  shoulder      3   86     3  26989   27   
23088  ERC_REST     ERC       REST  shoulder      4   94     2  27006    4   
46061  ERC_REST     ERC       REST  shoulder      3   72     3  27025   13   

       Year       Date  DOW  Weekday  Hour_Tot      Tot           Avg  
46075  2016 2016-03-27    6    False       122  3603145  29533.975410  
23088  2016 2016-04-04    0     True       122  3587428  29405.147541  
46061  2016 2016-03-13    6    False       122  3603145  29533.9754

## Case 3: Monthly, weekend/weekday, 24 hours (576 segments)
#### Metholodogy: Use groupby function to group by month, then weekend/weekday, then by 24 hours

In [6]:
case3_x = x2.copy()

aggregations = {x_column:['count',sum,'mean']}
case3 = case3_x.groupby(['Region','Month','Weekday','Hour'],as_index=False).agg(aggregations)
case3.columns = case3.columns.droplevel(0)
case3.columns = ['Region','Month','Weekday','Hour','Hour_Tot','Tot','Avg']
print(case1.head())
print('number of segments in dataset =',case3.shape[0]/reg_count)
case3.to_csv('../outputs/'+x_name+'/'+x_name+'_segments_2dt_mon_24hr.csv')
print()

case3_x2 = pd.merge(case3_x,case3,on=['Region','Month','Weekday','Hour'],how='left')
case3_x2 = case3_x2.sort_values(['Region',x_column])
print(case3_x2.head())
print('number of rows in dataset =',case3_x2.shape[0])
case3_x2.to_csv('../outputs/'+x_name+'/'+x_name+'_8760_2dt_mon_24hr.csv')

     Region  Month  Hour  Hour_Tot      Tot           Avg
0  ERC_REST      1     1        31  1042446  33627.290323
1  ERC_REST      1     2        31  1051661  33924.548387
2  ERC_REST      1     3        31  1079739  34830.290323
3  ERC_REST      1     4        31  1144492  36919.096774
4  ERC_REST      1     5        31  1239385  39980.161290
number of segments in dataset = 576.0

         Region R_Group R_Subgroup    Season  Month  DOY  Hour   Load  Day  \
46075  ERC_REST     ERC       REST  shoulder      3   86     3  26989   27   
23088  ERC_REST     ERC       REST  shoulder      4   94     2  27006    4   
46061  ERC_REST     ERC       REST  shoulder      3   72     3  27025   13   
46054  ERC_REST     ERC       REST  shoulder      3   65     3  27059    6   
23059  ERC_REST     ERC       REST  shoulder      3   65     2  27069    6   

       Year       Date  DOW  Weekday  Hour_Tot     Tot           Avg  
46075  2016 2016-03-27    6    False         8  222011  27751.375000  
23

## Case 4: Season, weekend/weekday, 24 hours (144 segments)
#### Methodology: groupby season, weekday, hour

In [7]:
case4_x = x2.copy()
#case4_seasons = pd.read_csv('inputs/season_bimonthly.csv')
#case4_seasons = case4_seasons.drop(['bimonthly'], axis=1)
#case4_seasons = case4_seasons.rename(columns={'seasonal':'Season_Group','month':'Month'})
#case4_x = pd.merge(case4_x, case4_seasons, on='Month', how='left')
#print(case4_x)

aggregations = {x_column:['count',sum,'mean']}
case4 = case4_x.groupby(['Region','Season','Weekday','Hour'],as_index=False).agg(aggregations)
case4.columns = case4.columns.droplevel(0)
case4.columns = ['Region','Season','Weekday','Hour','Hour_Tot','Tot','Avg']
print(case4.tail())
print('number of segments in dataset =',case4.shape[0]/reg_count)
case4.to_csv('../outputs/'+x_name+'/'+x_name+'_segments_2dt_sea_24hr.csv')
print()

case4_x2 = pd.merge(case4_x,case4,on=['Region','Season','Weekday','Hour'],how='left')
case4_x2 = case4_x2.sort_values(['Region',x_column])
print(case4_x2.head())
print('number of rows in dataset =',case4_x2.shape[0])
case4_x2.to_csv('../outputs/'+x_name+'/'+x_name+'_8760_2dt_sea_24hr.csv')

        Region  Season  Weekday  Hour  Hour_Tot     Tot          Avg
9067  WEC_SDGE  winter     True    20        63  190280  3020.317460
9068  WEC_SDGE  winter     True    21        63  182258  2892.984127
9069  WEC_SDGE  winter     True    22        63  167541  2659.380952
9070  WEC_SDGE  winter     True    23        63  149291  2369.698413
9071  WEC_SDGE  winter     True    24        63  133301  2115.888889
number of segments in dataset = 144.0

         Region R_Group R_Subgroup    Season  Month  DOY  Hour   Load  Day  \
46075  ERC_REST     ERC       REST  shoulder      3   86     3  26989   27   
23088  ERC_REST     ERC       REST  shoulder      4   94     2  27006    4   
46061  ERC_REST     ERC       REST  shoulder      3   72     3  27025   13   
46054  ERC_REST     ERC       REST  shoulder      3   65     3  27059    6   
23059  ERC_REST     ERC       REST  shoulder      3   65     2  27069    6   

       Year       Date  DOW  Weekday  Hour_Tot      Tot           Avg  
46075 

## 4 hour interval day types

In [3]:
#read in 4 hour interval counter
interval_4hr = pd.read_csv('inputs/interval_4hr.csv')
#print(interval_4hr)

x3 = pd.merge(x2,interval_4hr,on='Hour',how='left')
print(x3.tail())


         Region R_Group R_Subgroup  Season  Month  DOY  Hour  Load  Day  Year  \
551875  WECC_WY     WEC         WY  winter     12  361    24  1767   27  2011   
551876  WECC_WY     WEC         WY  winter     12  362    24  1744   28  2011   
551877  WECC_WY     WEC         WY  winter     12  363    24  1805   29  2011   
551878  WECC_WY     WEC         WY  winter     12  364    24  1791   30  2011   
551879  WECC_WY     WEC         WY  winter     12  365    24  1834   31  2011   

             Date  DOW  Weekday  4-hr  
551875 2011-12-27    1     True     6  
551876 2011-12-28    2     True     6  
551877 2011-12-29    3     True     6  
551878 2011-12-30    4     True     6  
551879 2011-12-31    5    False     6  


## Case 5: Monthly, single day type, 4 hour intervals (72 segments)
#### Methodology: use groupby by month, 4 hour intervals

In [9]:
case5_x = x3.copy()

aggregations = {x_column:['count',sum,'mean']}
case5 = case5_x.groupby(['Region','Month','4-hr'],as_index=False).agg(aggregations)
case5.columns = case5.columns.droplevel(0)
case5.columns = ['Region','Month','4-hr','Hour_Tot','Tot','Avg']
print(case5.head())
print('number of segments in dataset =',case5.shape[0]/reg_count)
case5.to_csv('../outputs/'+x_name+'/'+x_name+'_segments_1dt_mon_4hr.csv')
print()

case5_x2 = pd.merge(case5_x,case5,on=['Region','Month','4-hr'],how='left')
case5_x2 = case5_x2.sort_values(['Region',x_column])
print(case5_x2.head())
print('number of rows in dataset =',case5_x2.shape[0])
case5_x2.to_csv('../outputs/'+x_name+'/'+x_name+'_8760_1dt_mon_4hr.csv')

     Region  Month  4-hr  Hour_Tot      Tot           Avg
0  ERC_REST      1     1       124  4318338  34825.306452
1  ERC_REST      1     2       124  5025668  40529.580645
2  ERC_REST      1     3       124  4711362  37994.854839
3  ERC_REST      1     4       124  4504207  36324.250000
4  ERC_REST      1     5       124  4847894  39095.919355
number of segments in dataset = 72.0

         Region R_Group R_Subgroup    Season  Month  DOY  Hour   Load  Day  \
46075  ERC_REST     ERC       REST  shoulder      3   86     3  26989   27   
23088  ERC_REST     ERC       REST  shoulder      4   94     2  27006    4   
46061  ERC_REST     ERC       REST  shoulder      3   72     3  27025   13   
46054  ERC_REST     ERC       REST  shoulder      3   65     3  27059    6   
23059  ERC_REST     ERC       REST  shoulder      3   65     2  27069    6   

       Year       Date  DOW  Weekday  4-hr  Hour_Tot      Tot           Avg  
46075  2016 2016-03-27    6    False     1       124  3572927  2881

## Case 6: Bi-monthly weekend/weekday day-types, 4-hour intervals
#### Methodology: use groupby function and bimonthly groups

In [11]:
case6_x = x3.copy()
case6_bimonth = pd.read_csv('inputs/season_bimonthly.csv')
case6_bimonth = case6_bimonth.drop(['seasonal'], axis=1)
case6_bimonth = case6_bimonth.rename(columns={'bimonthly':'Bimonth','month':'Month'})

case6_x = pd.merge(case6_x, case6_bimonth, on='Month', how='left')

aggregations = {x_column:['count',sum,'mean']}
case6 = case6_x.groupby(['Region','Bimonth','Weekday','4-hr'],as_index=False).agg(aggregations)
case6.columns = case6.columns.droplevel(0)
case6.columns = ['Region','Bimonth','Weekday','4-hr','Hour_Tot','Tot','Avg']
print(case6.head())
print('number of segments in dataset =',case6.shape[0]/reg_count)
case6.to_csv('../outputs/'+x_name+'/'+x_name+'_segments_2dt_bim_4hr.csv')
print()

case6_x2 = pd.merge(case6_x,case6,on=['Region','Bimonth','Weekday','4-hr'],how='left')
case6_x2 = case6_x2.sort_values(['Region',x_column])
print(case6_x2.head())
print('number of rows in dataset =',case6_x2.shape[0])
case6_x2.to_csv('../outputs/'+x_name+'/'+x_name+'_8760_2dt_bim_4hr.csv')

     Region  Bimonth  Weekday  4-hr  Hour_Tot      Tot           Avg
0  ERC_REST        1    False     1        72  2362801  32816.680556
1  ERC_REST        1    False     2        72  2705068  37570.388889
2  ERC_REST        1    False     3        72  2622321  36421.125000
3  ERC_REST        1    False     4        72  2539521  35271.125000
4  ERC_REST        1    False     5        72  2729661  37911.958333
number of segments in dataset = 72.0

         Region R_Group R_Subgroup    Season  Month  DOY  Hour   Load  Day  \
46075  ERC_REST     ERC       REST  shoulder      3   86     3  26989   27   
23088  ERC_REST     ERC       REST  shoulder      4   94     2  27006    4   
46061  ERC_REST     ERC       REST  shoulder      3   72     3  27025   13   
46054  ERC_REST     ERC       REST  shoulder      3   65     3  27059    6   
23059  ERC_REST     ERC       REST  shoulder      3   65     2  27069    6   

       Year       Date  DOW  Weekday  4-hr  Bimonth  Hour_Tot      Tot  \
46075

## Case 7: Season-based months, weekend/weekday day-types, 4-hour intervals
#### Methodology: groupby function and applied season groups

In [4]:
case7_x = x3.copy()
case7_seasons = pd.read_csv('inputs/season_bimonthly.csv')
case7_seasons = case7_seasons.drop(['bimonthly'], axis=1)
case7_seasons = case7_seasons.rename(columns={'seasonal':'Season_Group','month':'Month'})

case7_x = pd.merge(case7_x, case7_seasons, on='Month', how='left')
#print(case4_load)

aggregations = {x_column:['count',sum,'mean']}
case7 = case7_x.groupby(['Region','Season_Group','Weekday','4-hr'],as_index=False).agg(aggregations)
case7.columns = case7.columns.droplevel(0)
case7.columns = ['Region','Season_Group','Weekday','4-hr','Hour_Tot','Tot','Avg']
print(case7.head())
print('number of segments in dataset =',case7.shape[0]/reg_count)
case7.to_csv('../outputs/'+x_name+'/'+x_name+'_segments_2dt_sgp_4hr.csv')
print()

case7_x2 = pd.merge(case7_x,case7,on=['Region','Season_Group','Weekday','4-hr'],how='left')
case7_x2 = case7_x2.sort_values(['Region',x_column])
print(case7_x2.head())
print('number of rows in dataset =',case7_x2.shape[0])
case7_x2.to_csv('../outputs/'+x_name+'/'+x_name+'_8760_2dt_sgp_4hr.csv')

     Region  Season_Group  Weekday  4-hr  Hour_Tot      Tot           Avg
0  ERC_REST             1    False     1       108  3533655  32719.027778
1  ERC_REST             1    False     2       108  3948984  36564.666667
2  ERC_REST             1    False     3       108  3994962  36990.388889
3  ERC_REST             1    False     4       108  3897223  36085.398148
4  ERC_REST             1    False     5       108  4166114  38575.129630
number of segments in dataset = 60.0

         Region R_Group R_Subgroup    Season  Month  DOY  Hour   Load  Day  \
46075  ERC_REST     ERC       REST  shoulder      3   86     3  26989   27   
23088  ERC_REST     ERC       REST  shoulder      4   94     2  27006    4   
46061  ERC_REST     ERC       REST  shoulder      3   72     3  27025   13   
46054  ERC_REST     ERC       REST  shoulder      3   65     3  27059    6   
23059  ERC_REST     ERC       REST  shoulder      3   65     2  27069    6   

       Year       Date  DOW  Weekday  4-hr  Seaso

## Case 8: Seasons, weekend/weekday day-types, 4-hour intervals
#### Methodology: groupby function and applied season groups

In [5]:
case8_x = x3.copy()

aggregations = {x_column:['count',sum,'mean']}
case8 = case8_x.groupby(['Region','Season','Weekday','4-hr'],as_index=False).agg(aggregations)
case8.columns = case8.columns.droplevel(0)
case8.columns = ['Region','Season','Weekday','4-hr','Hour_Tot','Tot','Avg']
print(case8.head())
print('number of segments in dataset =',case8.shape[0]/reg_count)
case8.to_csv('../outputs/'+x_name+'/'+x_name+'_segments_2dt_sea_4hr.csv')
print()

case8_x2 = pd.merge(case8_x,case8,on=['Region','Season','Weekday','4-hr'],how='left')
case8_x2 = case8_x2.sort_values(['Region',x_column])
print(case8_x2.head())
print('number of rows in dataset =',case8_x2.shape[0])
case8_x2.to_csv('../outputs/'+x_name+'/'+x_name+'_8760_2dt_sea_4hr.csv')

     Region    Season  Weekday  4-hr  Hour_Tot      Tot           Avg
0  ERC_REST  shoulder    False     1       140  4051780  28941.285714
1  ERC_REST  shoulder    False     2       140  4378835  31277.392857
2  ERC_REST  shoulder    False     3       140  4993309  35666.492857
3  ERC_REST  shoulder    False     4       140  5296142  37829.585714
4  ERC_REST  shoulder    False     5       140  5276131  37686.650000
number of segments in dataset = 36.0

         Region R_Group R_Subgroup    Season  Month  DOY  Hour   Load  Day  \
46075  ERC_REST     ERC       REST  shoulder      3   86     3  26989   27   
23088  ERC_REST     ERC       REST  shoulder      4   94     2  27006    4   
46061  ERC_REST     ERC       REST  shoulder      3   72     3  27025   13   
46054  ERC_REST     ERC       REST  shoulder      3   65     3  27059    6   
23059  ERC_REST     ERC       REST  shoulder      3   65     2  27069    6   

       Year       Date  DOW  Weekday  4-hr  Hour_Tot       Tot           

# Day type: Three (Weekday, Weekend, Peak Load)
## Find Peak Load Days in Each Month

In [14]:
#create temporary DF to find peak
load = load_dur.copy()

#groupby region, month, and day to sum the total day
aggregations1 = {'Load':sum}
md_sum = load.groupby(['Region','Month','Day'],as_index=False).agg(aggregations1)
md_sum.columns = ['Region','Month','Day','Load_MD_Tot']
#print(md_sum.tail())
print('number of rows in regional data =',md_sum.shape[0]/reg_count)

md_sum2 = pd.merge(load,md_sum,on=['Region','Month','Day'],how='left')
#print(md_sum2.tail())

#groupby region and month to find maximum 
aggregations2 = {'Load_MD_Tot':max}
md_max = md_sum2.groupby(['Region','Month'],as_index=False).agg(aggregations2)
md_max.columns = ['Region','Month','Load_MD_Max']
print(md_max[0:12])
print()

peakd = pd.merge(md_sum2,md_max,on=['Region','Month'],how='left')
peakd = peakd.drop(columns=['Unnamed: 0','Load'])
peakd = peakd.rename(columns={'Day':'DOY'})
print(peakd.head())


number of rows in regional data = 365.0
      Region  Month  Load_MD_Max
0   ERC_REST      1      1000605
1   ERC_REST      2       932895
2   ERC_REST      3       886217
3   ERC_REST      4       983152
4   ERC_REST      5      1083459
5   ERC_REST      6      1211630
6   ERC_REST      7      1267082
7   ERC_REST      8      1332321
8   ERC_REST      9      1231350
9   ERC_REST     10      1109240
10  ERC_REST     11      1004548
11  ERC_REST     12      1176963

     Region R_Group R_Subgroup  Season  Month  DOY  Hour  Load_MD_Tot  \
0  ERC_REST     ERC       REST  winter      1    1     1       917588   
1  ERC_REST     ERC       REST  winter      1    2     1       863132   
2  ERC_REST     ERC       REST  winter      1    3     1       934513   
3  ERC_REST     ERC       REST  winter      1    4     1       960576   
4  ERC_REST     ERC       REST  winter      1    5     1       916216   

   Load_MD_Max  
0      1000605  
1      1000605  
2      1000605  
3      1000605  
4     

In [15]:
x_peak = pd.merge(x2,peakd,on=['Region','R_Group','R_Subgroup','Season','Month','DOY','Hour'],how='left')
x_peak = x_peak.rename(columns={'Weekday':'Day_Type'})
#print(x_peak.tail())

#Return True if the load total equals the day identified as the max
x_peak.loc[x_peak['Day_Type'] == True, 'Day_Type'] = 'Weekday'
x_peak.loc[x_peak['Day_Type'] == False, 'Day_Type'] = 'Weekend'
x_peak.loc[x_peak['Load_MD_Tot'] == x_peak['Load_MD_Max'], 'Day_Type'] = 'Peak'
x_peak = x_peak.drop(['Load_MD_Tot','Load_MD_Max'], axis=1)
print(x_peak[0:15])


MemoryError: Unable to allocate 29.5 MiB for an array with shape (7, 551880) and data type int64

## Case 1: Monthly, Weekend/weekday/peak day-types, 24 hours (864 segments)
#### Methodology: similar to two day type, just adding in peak day types to sort by

In [9]:
case1_x = x_peak.copy()

aggregations = {x_column:['count',sum,'mean']}
case1 = case1_x.groupby(['Region','Month','Day_Type','Hour'],as_index=False).agg(aggregations)
case1.columns = case1.columns.droplevel(0)
case1.columns = ['Region','Month','Day_Type','Hour','Hour_Tot','Tot','Avg']
print(case1.head())
print('number of segments in dataset =',case1.shape[0]/reg_count)
case1.to_csv('../outputs/'+x_name+'/'+x_name+'_segments_3dt_mon_24hr.csv')
print()

case1_x2 = pd.merge(case1_x,case1,on=['Region','Month','Day_Type','Hour'],how='left')
case1_x2 = case1_x2.sort_values(['Region',x_column])
print(case1_x2.head(3))
print('number of rows in dataset =',case1_x2.shape[0])
case1_x2.to_csv('../outputs/'+x_name+'/'+x_name+'_8760_3dt_mon_24hr.csv')

     Region  Month Day_Type  Hour  Hour_Tot    Tot      Avg
0  ERC_REST      1     Peak     1         1  38567  38567.0
1  ERC_REST      1     Peak     2         1  39107  39107.0
2  ERC_REST      1     Peak     3         1  40311  40311.0
3  ERC_REST      1     Peak     4         1  43115  43115.0
4  ERC_REST      1     Peak     5         1  47186  47186.0
number of segments in dataset = 864.0

         Region R_Group R_Subgroup    Season  Month  DOY  Hour   Load  Day  \
46075  ERC_REST     ERC       REST  shoulder      3   86     3  26989   27   
23088  ERC_REST     ERC       REST  shoulder      4   94     2  27006    4   
46061  ERC_REST     ERC       REST  shoulder      3   72     3  27025   13   

       Year       Date  DOW Day_Type  Hour_Tot     Tot        Avg  
46075  2016 2016-03-27    6  Weekend         8  222011  27751.375  
23088  2016 2016-04-04    0  Weekday        20  579213  28960.650  
46061  2016 2016-03-13    6  Weekend         8  222011  27751.375  
number of rows i

## Case 2: Season, weekend/weekday/peak day-types, 24-hours (216 segments)

In [10]:
case2_x = x_peak.copy()

aggregations = {x_column:['count',sum,'mean']}
case2 = case2_x.groupby(['Region','Season','Day_Type','Hour'],as_index=False).agg(aggregations)
case2.columns = case2.columns.droplevel(0)
case2.columns = ['Region','Season','Day_Type','Hour','Hour_Tot','Tot','Avg']
print(case2.head())
print('number of segments in dataset =',case2.shape[0]/reg_count)
case2.to_csv('../outputs/'+x_name+'/'+x_name+'_segments_3dt_sea_24hr.csv')
print()

case2_x2 = pd.merge(case2_x,case2,on=['Region','Season','Day_Type','Hour'],how='left')
case2_x2 = case2_x2.sort_values(['Region',x_column])
print(case1_x2.head(3))
print('number of rows in dataset =',case1_x2.shape[0])
case2_x2.to_csv('../outputs/'+x_name+'/'+x_name+'_8760_3dt_sea_24hr.csv')

     Region    Season Day_Type  Hour  Hour_Tot     Tot       Avg
0  ERC_REST  shoulder     Peak     1         4  132690  33172.50
1  ERC_REST  shoulder     Peak     2         4  130802  32700.50
2  ERC_REST  shoulder     Peak     3         4  131751  32937.75
3  ERC_REST  shoulder     Peak     4         4  137910  34477.50
4  ERC_REST  shoulder     Peak     5         4  150015  37503.75
number of segments in dataset = 216.0

         Region R_Group R_Subgroup    Season  Month  DOY  Hour   Load  Day  \
46075  ERC_REST     ERC       REST  shoulder      3   86     3  26989   27   
23088  ERC_REST     ERC       REST  shoulder      4   94     2  27006    4   
46061  ERC_REST     ERC       REST  shoulder      3   72     3  27025   13   

       Year       Date  DOW Day_Type  Hour_Tot     Tot        Avg  
46075  2016 2016-03-27    6  Weekend         8  222011  27751.375  
23088  2016 2016-04-04    0  Weekday        20  579213  28960.650  
46061  2016 2016-03-13    6  Weekend         8  222011

## Case 3: Annual, weekend/weekday/peak day-types, 24-hours (72 segments)

In [11]:
case3_x = x_peak.copy()

aggregations = {x_column:['count',sum,'mean']}
case3 = case3_x.groupby(['Region','Day_Type','Hour'],as_index=False).agg(aggregations)
case3.columns = case3.columns.droplevel(0)
case3.columns = ['Region','Day_Type','Hour','Hour_Tot','Tot','Avg']
print(case3.head())
print('number of segments in dataset =',case3.shape[0]/reg_count)
case3.to_csv('../outputs/'+x_name+'/'+x_name+'_segments_3dt_ann_24hr.csv')
print()

case3_x2 = pd.merge(case3_x,case3,on=['Region','Day_Type','Hour'],how='left')
case3_x2 = case3_x2.sort_values(['Region',x_column])
print(case3_x2.head(3))
print('number of rows in dataset =',case3_x2.shape[0])
case3_x2.to_csv('../outputs/'+x_name+'/'+x_name+'_8760_3dt_ann_24hr.csv')

     Region Day_Type  Hour  Hour_Tot     Tot           Avg
0  ERC_REST     Peak     1        12  455893  37991.083333
1  ERC_REST     Peak     2        12  450804  37567.000000
2  ERC_REST     Peak     3        12  454384  37865.333333
3  ERC_REST     Peak     4        12  473371  39447.583333
4  ERC_REST     Peak     5        12  505999  42166.583333
number of segments in dataset = 72.0

         Region R_Group R_Subgroup    Season  Month  DOY  Hour   Load  Day  \
46075  ERC_REST     ERC       REST  shoulder      3   86     3  26989   27   
23088  ERC_REST     ERC       REST  shoulder      4   94     2  27006    4   
46061  ERC_REST     ERC       REST  shoulder      3   72     3  27025   13   

       Year       Date  DOW Day_Type  Hour_Tot      Tot           Avg  
46075  2016 2016-03-27    6  Weekend       104  3359100  32299.038462  
23088  2016 2016-04-04    0  Weekday       249  8176806  32838.578313  
46061  2016 2016-03-13    6  Weekend       104  3359100  32299.038462  
number 

## 4 hour interval

In [6]:
#read in 4 hour interval counter
#interval_4hr = pd.read_csv('inputs/interval_4hr.csv')
#print(interval_4hr.head())
#print(x_peak.head())

x_peak2 = pd.merge(x_peak,interval_4hr,on='Hour',how='left')
print(x_peak2.head(15))


      Region R_Group R_Subgroup  Season  Month  DOY  Hour   Load  Day  Year  \
0   ERC_REST     ERC       REST  winter      1    1     1  34807    1  2016   
1   ERC_REST     ERC       REST  winter      1    2     1  34716    2  2016   
2   ERC_REST     ERC       REST  winter      1    3     1  34736    3  2016   
3   ERC_REST     ERC       REST  winter      1    4     1  35914    4  2016   
4   ERC_REST     ERC       REST  winter      1    5     1  33845    5  2016   
5   ERC_REST     ERC       REST  winter      1    6     1  30995    6  2016   
6   ERC_REST     ERC       REST  winter      1    7     1  30186    7  2016   
7   ERC_REST     ERC       REST  winter      1    8     1  30745    8  2016   
8   ERC_REST     ERC       REST  winter      1    9     1  38702    9  2016   
9   ERC_REST     ERC       REST  winter      1   10     1  38567   10  2016   
10  ERC_REST     ERC       REST  winter      1   11     1  37495   11  2016   
11  ERC_REST     ERC       REST  winter      1   12 

## Case 4: Bi-monthly, weekend/weekday/peak day-types, 4-hour intervals

In [7]:
case4_x = x_peak2.copy()
case4_bimonth = pd.read_csv('inputs/season_bimonthly.csv')
case4_bimonth = case4_bimonth.drop(['seasonal'], axis=1)
case4_bimonth = case4_bimonth.rename(columns={'bimonthly':'Bimonth','month':'Month'})

case4_x = pd.merge(case4_x, case4_bimonth, on='Month', how='left')

aggregations = {x_column:['count',sum,'mean']}
case4 = case4_x.groupby(['Region','Bimonth','Day_Type','4-hr'],as_index=False).agg(aggregations)
case4.columns = case4.columns.droplevel(0)
case4.columns = ['Region','Bimonth','Day_Type','4-hr','Hour_Tot','Tot','Avg']
print(case4.head())
print('number of segments in dataset =',case4.shape[0]/reg_count)
case4.to_csv('../outputs/'+x_name+'/'+x_name+'_segments_3dt_bim_4hr.csv')
print()

case4_x2 = pd.merge(case4_x,case4,on=['Region','Bimonth','Day_Type','4-hr'],how='left')
case4_x2 = case4_x2.sort_values(['Region',x_column])
print(case4_x2.head())
print('number of rows in dataset =',case4_x2.shape[0])
case4_x2.to_csv('../outputs/'+x_name+'/'+x_name+'_8760_3dt_bim_4hr.csv')

     Region  Bimonth Day_Type  4-hr  Hour_Tot     Tot        Avg
0  ERC_REST        1     Peak     1         8  312443  39055.375
1  ERC_REST        1     Peak     2         8  366535  45816.875
2  ERC_REST        1     Peak     3         8  322104  40263.000
3  ERC_REST        1     Peak     4         8  297348  37168.500
4  ERC_REST        1     Peak     5         8  332533  41566.625
number of segments in dataset = 108.0

         Region R_Group R_Subgroup    Season  Month  DOY  Hour   Load  Day  \
46075  ERC_REST     ERC       REST  shoulder      3   86     3  26989   27   
23088  ERC_REST     ERC       REST  shoulder      4   94     2  27006    4   
46061  ERC_REST     ERC       REST  shoulder      3   72     3  27025   13   
46054  ERC_REST     ERC       REST  shoulder      3   65     3  27059    6   
23059  ERC_REST     ERC       REST  shoulder      3   65     2  27069    6   

       Year       Date  DOW Day_Type  4-hr  Bimonth  Hour_Tot      Tot  \
46075  2016 2016-03-27    6 

## Case 5: Season-based months, weekend/weekday/peak day-types, 4-hour intervals

In [8]:
case5_x = x_peak2.copy()
case5_seasons = pd.read_csv('inputs/season_bimonthly.csv')
case5_seasons = case5_seasons.drop(['bimonthly'], axis=1)
case5_seasons = case5_seasons.rename(columns={'seasonal':'Season_Group','month':'Month'})

case5_x = pd.merge(case5_x, case5_seasons, on='Month', how='left')

aggregations = {x_column:['count',sum,'mean']}
case5 = case5_x.groupby(['Region','Season_Group','Day_Type','4-hr'],as_index=False).agg(aggregations)
case5.columns = case5.columns.droplevel(0)
case5.columns = ['Region','Season_Group','Day_Type','4-hr','Hour_Tot','Tot','Avg']
print(case5.head())
print('number of segments in dataset =',case5.shape[0]/reg_count)
case5.to_csv('../outputs/'+x_name+'/'+x_name+'_segments_3dt_sgp_4hr.csv')
print()

case5_x2 = pd.merge(case5_x,case5,on=['Region','Season_Group','Day_Type','4-hr'],how='left')
case5_x2 = case5_x2.sort_values(['Region',x_column])
print(case5_x2.head())
print('number of rows in dataset =',case5_x2.shape[0])
case5_x2.to_csv('../outputs/'+x_name+'/'+x_name+'_8760_3dt_sgp_4hr.csv')

     Region  Season_Group Day_Type  4-hr  Hour_Tot     Tot           Avg
0  ERC_REST             1     Peak     1        12  504276  42023.000000
1  ERC_REST             1     Peak     2        12  583360  48613.333333
2  ERC_REST             1     Peak     3        12  527061  43921.750000
3  ERC_REST             1     Peak     4        12  478413  39867.750000
4  ERC_REST             1     Peak     5        12  530620  44218.333333
number of segments in dataset = 90.0

         Region R_Group R_Subgroup    Season  Month  DOY  Hour   Load  Day  \
46075  ERC_REST     ERC       REST  shoulder      3   86     3  26989   27   
23088  ERC_REST     ERC       REST  shoulder      4   94     2  27006    4   
46061  ERC_REST     ERC       REST  shoulder      3   72     3  27025   13   
46054  ERC_REST     ERC       REST  shoulder      3   65     3  27059    6   
23059  ERC_REST     ERC       REST  shoulder      3   65     2  27069    6   

       Year       Date  DOW Day_Type  4-hr  Season_Grou

## Case 6: Season, weekend/weekday/peak day-types, 4-hour intervals

In [9]:
case6_x = x_peak2.copy()

aggregations = {x_column:['count',sum,'mean']}
case6 = case6_x.groupby(['Region','Season','Day_Type','4-hr'],as_index=False).agg(aggregations)
case6.columns = case6.columns.droplevel(0)
case6.columns = ['Region','Season','Day_Type','4-hr','Hour_Tot','Tot','Avg']
print(case6.head())
print('number of segments in dataset =',case6.shape[0]/reg_count)
case6.to_csv('../outputs/'+x_name+'/'+x_name+'_segments_3dt_sea_4hr.csv')
print()

case6_x2 = pd.merge(case6_x,case6,on=['Region','Season','Day_Type','4-hr'],how='left')
case6_x2 = case6_x2.sort_values(['Region',x_column])
print(case6_x2.head())
print('number of rows in dataset =',case6_x2.shape[0])
case6_x2.to_csv('../outputs/'+x_name+'/'+x_name+'_8760_3dt_sea_4hr.csv')

     Region    Season Day_Type  4-hr  Hour_Tot     Tot         Avg
0  ERC_REST  shoulder     Peak     1        16  533153  33322.0625
1  ERC_REST  shoulder     Peak     2        16  619476  38717.2500
2  ERC_REST  shoulder     Peak     3        16  707544  44221.5000
3  ERC_REST  shoulder     Peak     4        16  785332  49083.2500
4  ERC_REST  shoulder     Peak     5        16  743101  46443.8125
number of segments in dataset = 54.0

         Region R_Group R_Subgroup    Season  Month  DOY  Hour   Load  Day  \
46075  ERC_REST     ERC       REST  shoulder      3   86     3  26989   27   
23088  ERC_REST     ERC       REST  shoulder      4   94     2  27006    4   
46061  ERC_REST     ERC       REST  shoulder      3   72     3  27025   13   
46054  ERC_REST     ERC       REST  shoulder      3   65     3  27059    6   
23059  ERC_REST     ERC       REST  shoulder      3   65     2  27069    6   

       Year       Date  DOW Day_Type  4-hr  Hour_Tot      Tot           Avg  
46075  2016 2